<a href="https://colab.research.google.com/github/TKph/colab/blob/main/%E5%8B%BE%E9%85%8D%E3%83%96%E3%83%BC%E3%82%B9%E3%83%86%E3%82%A3%E3%83%B3%E3%82%B0%E6%B1%BA%E5%AE%9A%E6%9C%A8%E3%81%AB%E3%82%88%E3%82%8B%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---

# データの生成

---



In [ ]:
from sklearn import datasets, ensemble
from sklearn.model_selection import train_test_split

In [ ]:
X, y = datasets.make_classification(n_classes=2, n_features=20, n_samples=500, random_state=0)  #2クラス, 特徴量は20個
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)



---


# 勾配ブースティング決定木の適用

---



In [ ]:
gb = ensemble.GradientBoostingClassifier(n_estimators=500, random_state=0)

#比較のためにランダムフォレストも用意する
rf = ensemble.RandomForestClassifier(n_estimators=500, random_state=0)

gb.fit(X_train, y_train)
rf.fit(X_train, y_train)

print(f'Gradient Boosting Train score: {gb.score(X_train, y_train)}')
print(f'Random Forest Train score: {gb.score(X_train, y_train)}')
print(f'Gradient Boosting Train score: {gb.score(X_test, y_test)}')
print(f'Random Forest Train score: {gb.score(X_test, y_test)}')

Gradient Boosting Train score: 1.0
Random Forest Train score: 1.0
Gradient Boosting Train score: 0.912
Random Forest Train score: 0.912


In [ ]:
#early stoppingを適用
gbes = ensemble.GradientBoostingClassifier(n_estimators=500, validation_fraction=0.25, n_iter_no_change=5, random_state=0)

gbes.fit(X_train, y_train)

print(f'GB with early stopping Train score: {gbes.score(X_train, y_train)}')
print(f'GB with early stopping Test score: {gbes.score(X_test, y_test)}')
print(f'the number of estimators: {len(gbes.estimators_)}')

GB with early stopping Train score: 0.9653333333333334
GB with early stopping Test score: 0.92
the number of estimators: 44




---

# 解説

---



まず, 決定木の発展アルゴリズムは他にもAdaBoostやXGBoost, Light GBMなどがある.

数式を使った説明はYoutubeに公開されている"Gradient Boost Part3: Classification"や"Introducsion to Boosted Trees"に譲る.

簡単のために2クラス分類とする.


1.   分類確率と残差を計算する
2.   木を構築する
3.   木の葉の予測値を計算する
4.   予測値を用いて分類確率と残差を再計算する
5.   残差を最小にするような木を構築する
6.   繰り返す



1.

それぞれのデータに対して分類確率を計算する. 最初は $分類確率p_0=\frac{クラス0番}{データ数}$とする. また, $残差(誤差)e=正解ラベル-分類確率$である. 誤差といえば2乗や絶対値を取るのがよくある操作であるが, このアルゴリズムでは必要ない. 2クラス分類の場合, 正解ラベルは0または1であろう.

2.

ここの木は何の指標によって構築されるのか分からない

3.

作成した木のそれぞれの葉(ノード)に対して予測値を計算する. $x$をそのノードに属するデータだとする.
$$
予測値\gamma = \frac{\sum_X e_x}{\sum_Xe_xp_x}
$$
予測値は正しく分類できている葉では正, できていない葉では負になることが多い.

4.

分類確率を次の式によって更新する.
$$
p = p_0 + \eta\gamma
$$
正しく分類できているデータの分類確率は大きくなり, できていないデータのそれは小さくなると考えられる.

5.

2乗和誤差を最小にするように木を構築する.どこの二乗和誤差??